In [7]:
import os,sys,json,re
import pandas as pd

#### Notes:
The file directory in which the .rec files are located is named 'day' and needs to be within the same folder as this notebook.

#### Integrity Check of Data Files

In [28]:
file_list = os.listdir('day')
r0 = re.compile(r'^.*.rec$')
r1 = re.compile(r'^[0-9]{3}-(?P<day>day[0-9]+).rec$')
day = ''
flag = False
for file in file_list: #assume file is file name
    if r0.match(file): #if our .rec regex matches - we know it's a .rec file
        flag = True
        results = re.search(r1, file) #if no match results = None
        if not results:
            print("incorrectly formatted .rec")
            print(file)
            break
        else:
            if day == '':
                day = results.group('day') #gets day
            else:
                if results.group('day') != day:
                    print("incorrect day")
                    break

In [33]:
rec_list = [rec for rec in file_list if r0.match(rec)]
rec_list = rec_list.sort()

#### DataFrame Creation

In [69]:
json_list = []
with open('day/{}'.format(rec_list[1])) as f:
    for r in f.readlines():
        json_list.append(json.loads(r))

In [126]:
timestamps,uuids,pos_xs,pos_ys,neighbor_xs,neighbor_ys,phases,last_rec = [],[],[],[],[],[],[],[]
nano = 0
for rec in rec_list:
    for l in json_list:
        if l['message'] == '_SAVE_LIVEFRAME':
            for visual in l['data']['visuals']:
                if 'uuid' in visual and visual.get('type') == 'Mesh':
                    uuids.append(visual['uuid'])
                    timestamps.append(l['timestamp'] + nano)
                    pos_xs.append(float(visual['position']['x']))
                    pos_ys.append(float(visual['position']['y']))
                    neighbor_xs.append(float(visual['dashedLine']['x2']))
                    neighbor_ys.append(float(visual['dashedLine']['y2']))
                    phases.append(visual['dashedLine']['settings']['width'])
                    last_rec.append(0)
    last_rec[-1] = 1
    nano = timestamps[-1]
df_dict = {'uuids': uuids, 'timestamps':timestamps, 'pos_xs':pos_xs, 'pos_ys':pos_ys, 'neighbor_xs':neighbor_xs, 'neighbor_ys':neighbor_ys, 'phases':phases}
df = pd.DataFrame(df_dict)

In [128]:
df

,neighbor_xs,neighbor_ys,phases,pos_xs,pos_ys,timestamps,uuids
0,-0.0404,0.258,1,0.1490,-1.450,41550280,A943B4A5-88AC-4E26-83C4-E2F78CD9D1E1
1,0.1490,-1.450,15,-0.0404,0.258,41550280,B43FDA85-832B-44D2-A8A6-5C0E75C06FE1
2,0.1820,0.218,15,0.0134,-1.520,143818794,A943B4A5-88AC-4E26-83C4-E2F78CD9D1E1
3,0.0134,-1.520,15,0.1820,0.218,143818794,B43FDA85-832B-44D2-A8A6-5C0E75C06FE1
4,0.5440,0.115,15,-0.1220,-1.600,244045742,A943B4A5-88AC-4E26-83C4-E2F78CD9D1E1
5,-0.1220,-1.600,1,0.5440,0.115,244045742,B43FDA85-832B-44D2-A8A6-5C0E75C06FE1
6,0.8020,0.030,15,-0.2920,-1.670,344189521,A943B4A5-88AC-4E26-83C4-E2F78CD9D1E1
7,-0.2920,-1.670,1,0.8020,0.030,344189521,B43FDA85-832B-44D2-A8A6-5C0E75C06FE1
8,1.0800,-0.179,15,-0.4910,-1.750,444909701,A943B4A5-88AC-4E26-83C4-E2F78CD9D1E1
9,-0.4910,-1.750,1,1.0800,-0.179,444909701,B43FDA85-832B-44D2-A8A6-5C0E75C06FE1
